# Рекомендация тарифов

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## Изучим файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')


In [3]:
print(df.shape)
df.head(5)


(3214, 5)


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

## Разобьем данные на выборки

In [5]:
features = df.drop('is_ultra', axis=1)
target = df.is_ultra

features_train, features_tmp, target_train, target_tmp = \
                train_test_split(features,\
                                 target,\
                                 test_size=0.4,\
                                 stratify=target)
features_test, features_valid, target_test, target_valid = \
                train_test_split(features_tmp,\
                                 target_tmp,\
                                 test_size=0.5,
                                 stratify=target_tmp)

## Исследуем модели

In [6]:
random_state = 12345

### Descision Tree Classifier

In [7]:
best_acc = 0

for criterion in ['gini', 'entropy']:
    for depth in range(1, 20):
        model = DecisionTreeClassifier(criterion=criterion,\
                                       max_depth=depth,\
                                       random_state=random_state)
        
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        
        accuracy = accuracy_score(predictions, target_valid)
        
        if accuracy > best_acc:
            model_Tree = model
            best_acc = accuracy
            best_criterion = criterion
            best_depth = depth
            
print(best_acc, best_criterion, best_depth)
        

0.7931570762052877 gini 3


Лучшая точность дерева достигается при максимальной глубине в 19 нод, и энтропии в качестве критерия.

### Random Forest Classifier

In [8]:
best_acc = 0

for criterion in ['gini', 'entropy']:
    for depth in range(1,11):
        for estim in range(1, 41):
            model = RandomForestClassifier(criterion=criterion,\
                                           max_depth=depth,\
                                           n_estimators=estim,\
                                           random_state=random_state)

            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)

            accuracy = accuracy_score(predictions, target_valid)

            if accuracy > best_acc:
                model_Forest = model
                best_acc = accuracy
                best_criterion = criterion
                best_depth = depth
                best_estim_numb = estim
            
print(best_acc, best_criterion, best_depth, best_estim_numb)



0.8102643856920684 entropy 10 27


При случайном лесе - лучшая точность достигается с Критерием джини, глубине 10, и числе деревьев 27 .

### Linear Regression

In [9]:
best_acc = 0

for solv in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:

    model = LogisticRegression(random_state=random_state)

    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)

    accuracy = accuracy_score(predictions, target_valid)
    if accuracy > best_acc:
        model_Reg = model
        best_acc = accuracy
        solver = solv

print(best_acc, solv)

0.7060653188180405 saga


Лучшая точность на тестовых значениях сильно ниже, чем у деревьев и достигается при saga solver.

## Тестирование лучшей модели

In [10]:
pred_Tree = model_Tree.predict(features_test)
pred_Forest = model_Forest.predict(features_test)
pred_Reg = model_Reg.predict(features_test)

data = pd.DataFrame.from_records(list(zip(pred_Tree, \
                                          pred_Forest, \
                                          pred_Reg, \
                                          target_test)), \
                                 columns=['Tree', \
                                          'Forest', \
                                          'Reg', \
                                          'Target'])

print(accuracy_score(data.Tree, data.Target),\
      accuracy_score(data.Forest, data.Target),\
      accuracy_score(data.Reg, data.Target))



0.80248833592535 0.8087091757387247 0.713841368584759


На тестовой выборке лучшая точность достигается при модели случайного леса, но точности уже отличаются не так сильно.

## (бонус) Проверим модели на адекватность

In [11]:
adeq = pd.Series([0] * 643)

accuracy_score(target_test, adeq)


0.6936236391912908

In [12]:
adeq_1 = pd.Series([1] * 643)

accuracy_score(target_test, adeq_1)


0.30637636080870917

## Вывод

На основе данных клиентов оператора сотовой связи удалось проанализировать поведение клиентов и обучить модель для поиска оптимального тарифа.